# Tugas Besar 1 IF3270 Pembelajaran Mesin <br /> Feedforward Neural Network

## Kelompok 39

- Dzaky Satrio Nugroho - 13522059
- Julian Caleb Simandjuntak - 13522099
- Rafiki Prawhira Harianto - 13522065

In [ ]:
# Import dulu
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import ffnn
import importlib
import pickle

## IMPORT DATASET

In [ ]:
# Load datasetnya dulu min
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42)

## FFNN TUBES

In [ ]:
# Konvert dulu karena aselinya string
y_int = y.astype(int).reshape(-1, 1) 

# Kita one hot encoding sejenak
encoder = OneHotEncoder(sparse_output=False) 
y_one_hot = encoder.fit_transform(y_int)

# Split the dataset
X_train_tubes, X_test_tubes, y_train_tubes, y_test_tubes = train_test_split(X, y_one_hot, test_size=10000, random_state=42)

In [ ]:
# Bikin modelnya gan
importlib.reload(ffnn)
model = ffnn.FFNN(
    layers = [784, 3, 3, 10], # Input harus 784, output harus 10
    activations = ["sigmoid", "sigmoid", "sigmoid"],
    loss = "mse",
    initialization = "uniform",
    seed=42,
    learning_rate = 0.01,
    epochs = 3,
    regularization=None, 
    lambda_reg=0.1,
    normalization="rmsnorm",
    verbose = 1
)

# Kita train sejenak
model.train(X_train_tubes, y_train_tubes, val_split=0.2)
model.plot_loss()
model.visualize_weights(start=1, end=None, display_size=5)
model.visualize_gradient_weights(start=1, end=None, display_size=5)

In [ ]:
# Prediksi skor gan
y_pred_tubes = model.predict(X_test_tubes)

result = np.zeros_like(y_pred_tubes)
for i, row in enumerate(y_pred_tubes):
    max_index = np.argmax(row)
    result[i, max_index] = 1
y_pred_tubes = result

y_pred_classes = np.argmax(y_pred_tubes, axis=1)
y_test_classes = np.argmax(y_test_tubes, axis=1)
    
correct_predictions = np.sum(y_pred_classes == y_test_classes)
total_samples = y_test_tubes.shape[0]
    
accuracy = correct_predictions / total_samples
print(f"Akurasi: {accuracy:.2f}")

## FFNN Sklearn

In [ ]:
# Train test split dulu
X_train_sklearn, X_test_sklearn, y_train_sklearn, y_test_sklearn = train_test_split(X, y_one_hot, test_size=10000, random_state=42)

In [ ]:
# Pucuk ubi pucuk kangkung, bikin modelnya dulu bung
model = MLPClassifier(
    hidden_layer_sizes=(3, 3, 2),
    activation='logistic',
    solver='adam',
    max_iter=500,
    learning_rate='constant',
    random_state=42
)

In [ ]:
# Train model
model.fit(X_train_sklearn, y_train_sklearn)

In [ ]:
# Jajal prediksi, cek akurasi
y_pred = model.predict(X_test_sklearn)
accuracy = accuracy_score(y_test_sklearn, y_pred)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
# Visualisasikan loss
plt.plot(model.loss_curve_)
plt.title("Loss Curve")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()